In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = '/content/drive/MyDrive/ML_final_project'
assert FOLDERNAME is not None, "[!] Enter the foldername."

DATAPATH = f'{FOLDERNAME}/data/'


Mounted at /content/drive


In [ ]:
!pip install numpy==1.26.4 --force-reinstall -q
!pip install mlflow==2.15.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 119.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timedelta

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
#import optuna
import joblib
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from sklearn.pipeline import Pipeline

In [ ]:
!pip install dagshub mlflow -q


import dagshub
import mlflow


dagshub.init(repo_owner='skara-21', repo_name='ML_Final_Project', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/skara-21/ML_Final_Project.mlflow')

try:
    experiment_id = mlflow.create_experiment("XGBoost_Training")
    print(f"Created new experiment: XGBoost_Training")
except mlflow.exceptions.MlflowException:
    experiment = mlflow.get_experiment_by_name("XGBoost_Training")
    experiment_id = experiment.experiment_id
    print(f"Using existing experiment: XGBoost_Training")

mlflow.set_experiment("XGBoost_Training")

print(" MLflow setup complete")
print(" Your experiments will be visible at:")
print("   https://dagshub.com/kechik21/ML_Final_Project")

# **MLflow Setup**

In [ ]:
#mlflow.set_tracking_uri("file:///content/drive/MyDrive/ML_final_project/mlruns")
#experiment_name = "XGBoost_Training"

# Create or get experiment
#try:
 #   experiment_id = mlflow.create_experiment(experiment_name)
 #   print(f"Created new experiment: {experiment_name}")
#except mlflow.exceptions.MlflowException:
 #   experiment = mlflow.get_experiment_by_name(experiment_name)
 #   experiment_id = experiment.experiment_id
 #   print(f"Using existing experiment: {experiment_name}")

#mlflow.set_experiment(experiment_name)
#print(f"Experiment ID: {experiment_id}")

**Initial Data**

In [ ]:
train_df = pd.read_csv(DATAPATH + 'train.csv')
test_df = pd.read_csv(DATAPATH + 'test.csv')
features_df = pd.read_csv(DATAPATH + 'features.csv')
stores_df = pd.read_csv(DATAPATH + 'stores.csv')


In [ ]:
print(f"Train Shape : {train_df.shape}")
print(f"Train Shape : {test_df.shape}")
print(f"Train Shape : {features_df.shape}")
print(f"Train Shape : {stores_df.shape}")

დავლოგავ ჩემი საწყისი დეითას ზომებს, რათა შემდეგ ქლინინგის შედეგად მიღებულს შევადარო

In [ ]:
with mlflow.start_run(run_name= "XGBoost_Data_Loading") as run:
  mlflow.log_metric("initial_train_rows", train_df.shape[0])
  mlflow.log_metric("initial_train_cols", train_df.shape[1])
  mlflow.log_metric("initial_test_rows", test_df.shape[0])
  mlflow.log_metric("initial_test_cols", test_df.shape[1])


# **Cleaning**

In [ ]:
from datetime import datetime, timedelta
#იმისთვის, რომ ბევრად მარტივი გახდეს მუშაობა და
#ადვილად შევძლოთ დროსთან კავშირი ,გადავიყვანოთ
#object ---> datetime

def object_to_datetime(input, col = 'Data', curr_part=''):
  before_type = input[col].dtype
  input[col]=pd.to_datetime(input[col])
  after_type = input[col].dtype
  print(f"Converting data types for {curr_part}" )
  print(f"Before - {before_type}")
  print(f"After - {after_type}")
  print(" ")
  return input

train_df = object_to_datetime(train_df, col='Date', curr_part='Train data')
test_df = object_to_datetime(test_df, col='Date', curr_part='Test data')
features_df = object_to_datetime(features_df, col='Date', curr_part='Features data')


In [ ]:
#დამატებითი დეითას მერჯინგი ტესტთან და ტრეინთან
train_merged = train_df.merge(stores_df, on='Store', how='left')
test_merged = test_df.merge(stores_df, on='Store', how='left')

#features-ის დამატება
train_merged = train_merged.merge(features_df, on=['Store', 'Date'], how='left')
test_merged = test_merged.merge(features_df, on=['Store', 'Date'], how='left')

In [ ]:
def group_columns_by_dtype(df):
    data_types = df.dtypes.unique()
    for dtype in data_types:
        cols = df.select_dtypes(include=[dtype]).columns.tolist()
        print(f"\n{dtype} columns:")
        for col in cols:
            print(f" * {col}")


In [ ]:
group_columns_by_dtype(train_merged)

In [ ]:
def print_columns_info():
    print("Feature Columns:")
    print("   ", features_df.columns.tolist(), "\n")

    print("Store Columns:")
    print("   ", stores_df.columns.tolist(), "\n")

    print("Train Columns:")
    print("   Before merging:", train_df.columns.tolist())
    print("   After  merging:", train_merged.columns.tolist(), "\n")

    print("Test Columns:")
    print("   Before merging:", test_df.columns.tolist())
    print("   After  merging:", test_merged.columns.tolist(), "\n")


print_columns_info()


გაგვიჩნდა პატარა პრობლემა, მერჯინგის დროს გაჩნდა IsHoliday_x & IsHoliday_y , ამიტომაც გავარკვიოთ რომლის დატოვე სჯობს

In [ ]:
same_cols = (train_merged['IsHoliday_x'] == train_merged['IsHoliday_y']).all()
print("Are they the same? ", same_cols)


In [ ]:
train_merged['IsHoliday'] = train_merged['IsHoliday_y'].fillna(train_merged['IsHoliday_x'])
test_merged['IsHoliday'] = test_merged['IsHoliday_y'].fillna(test_merged['IsHoliday_x'])

train_merged = train_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
test_merged = test_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

რადგანაც Weekly Sales ზოგიერთგან უარყოფითი გვაქვს, მოვიშოროთ

In [ ]:
init_rows = len(train_merged)
train_merged = train_merged[train_merged['Weekly_Sales'] >=0 ]
removed_ones = init_rows - len(train_merged)
print(f"Removed negative values are {removed_ones}")

In [ ]:
with mlflow.start_run(run_name="XGBoost_Cleaning") as run:
  mlflow.log_metric("negative_saled_removed",removed_ones)


**Finding Missing Values**

In [ ]:
def check_missing(input, name="DataFrame"):
    missing_amount = input.isnull().sum()
    really_missing = missing_amount[missing_amount > 0]

    if not really_missing.empty:
        print(f" Missing values in {name}:")
        print(really_missing)
        return really_missing.to_dict()
    else:
        print(f"No missing values")
        return {}

In [ ]:
with mlflow.start_run(run_name="XGBoost_Missing_values") as run:
   train_missing = check_missing(train_merged, "Train Merged")
   test_missing = check_missing(test_merged, "Test Merged")

   #დავლოგოთ მისინგები
   for col, count in train_missing.items():
        mlflow.log_metric(f"train_missing_{col}", count)
   for col, count in test_missing.items():
        mlflow.log_metric(f"test_missing_{col}", count)

In [ ]:
check_missing(train_merged, "Train Merged")
check_missing(test_merged, "Test Merged")

In [ ]:
num_cols = ['CPI','Fuel_Price','Unemployment','Temperature']

In [ ]:
def fill_numericals(num_cols,train_merged,test_merged):
  for col in num_cols:
    if col in train_merged.columns:
        train_missing = train_merged[col].isnull().sum()
        if train_missing > 0:
            print(f"Filling {train_missing} missing values in {col} (train)")
            train_merged[col] = train_merged.groupby('Store')[col].ffill()
            train_merged[col] = train_merged[col].fillna(train_merged[col].median())


        test_missing = test_merged[col].isnull().sum()
        if test_missing > 0:
            print(f"Filling {test_missing} missing values in {col} (test)")
            test_merged[col] = test_merged.groupby('Store')[col].ffill()
            test_merged[col] = test_merged[col].fillna(test_merged[col].median())

In [ ]:
fill_numericals(num_cols,train_merged,test_merged)

რადგანაც Mark Downs გვაქვს მხოლოდ missing data-თი, შევავსოთ ნულებით

In [ ]:
markdowns = [col for col in train_merged.columns if 'MarkDown' in col]
for col in markdowns:
    train_merged[col] = train_merged[col].fillna(0)
    test_merged[col] = test_merged[col].fillna(0)

ყოველი შემთხვევისთვის, isHoliday-ც გავითვალისწინოთ

In [ ]:
train_merged['IsHoliday'] = train_merged['IsHoliday'].fillna(False)
test_merged['IsHoliday'] = test_merged['IsHoliday'].fillna(False)

Markdown-ების ნაწილი, რომელიც შედარებით ცუდია, კერძოს 1/3/5 მოვიშოროთ

In [ ]:
removed_cols = ['MarkDown1','MarkDown3','MarkDown5']
mlflow.log_param("removed_cols",removed_cols)

In [ ]:
train_merged = train_merged.drop('MarkDown1', axis=1)
train_merged = train_merged.drop('MarkDown3', axis=1)
train_merged = train_merged.drop('MarkDown5', axis=1)
test_merged = test_merged.drop('MarkDown1', axis=1)
test_merged = test_merged.drop('MarkDown3', axis=1)
test_merged = test_merged.drop('MarkDown5', axis=1)

# **Feature Engineering**

In [ ]:
def additional_time_feat(input):
  input = input.copy()
  input['Day'] = input['Date'].dt.day.astype('int64')
  input['Week'] = input['Date'].dt.isocalendar().week.astype('int64')
  input['Month'] = input['Date'].dt.month.astype('int64')
  input['Year'] = input['Date'].dt.year.astype('int64')
  const_month = 2 * np.pi * input['Month']
  const_week = 2 * np.pi * input['Week']
  input['Month_sin'] = np.sin( const_month/ 12).astype('float64')
  input['Month_cos'] = np.cos(const_month / 12).astype('float64')
  input['Week_sin'] = np.sin( const_week/ 52).astype('float64')
  input['Week_cos'] = np.cos(const_week/ 52).astype('float64')

  return input

In [ ]:
def additional_cat_feat(input):
  input= input.copy()
  label_enc_type = LabelEncoder()
  input['Type_le'] = label_enc_type.fit_transform(input['Type'])
  return input, label_enc_type

In [ ]:
def additional_month_feat_train(input, col='Weekly_Sales'):
    input = input.copy()
    input = input.reset_index(drop=True)
    input = input.sort_values(['Store','Dept','Date'])
    print(f"Additional features are being added for {col}")

    my_w = [4, 8, 12]

    for w in my_w:
        print(f"Adding {w} week rolling mean features")
        added_mn = []
        for (store, dept), group in input.groupby(['Store', 'Dept']):
            group = group.sort_values('Date').reset_index(drop=True)
            month_mean = group[col].rolling(w, min_periods=1).mean()
            added_mn.extend(month_mean.values)
        input = input.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)
        input[f'{col}_month_mean_{w}'] = added_mn
        print(f"Added column: {col}_month_mean_{w}")
    print("All addition is done")
    return input

In [ ]:
def additional_month_feat_test(train_data, test_input, col='Weekly_Sales'):
    test_input = test_input.copy()
    print(f"Adding rolling features to test data depending on {col}")
    month_cols = [c for c in train_data.columns if f'{col}_month_mean_' in c]
    for month_col in month_cols:
        print(f"Adding {month_col} to test data")
        store_dept_avg = train_data.groupby(['Store', 'Dept'])[month_col].mean().reset_index()
        store_dept_avg.columns = ['Store', 'Dept', month_col]
        test_input = test_input.merge(store_dept_avg, on=['Store', 'Dept'], how='left')
        overall_avg = train_data[month_col].mean()
        test_input[month_col] = test_input[month_col].fillna(overall_avg)

    print("All rolling features are complete")
    return test_input

In [ ]:
mlflow.end_run()

with mlflow.start_run(run_name="XGBoost_Feature_Engineering") as run:
  train_merged= additional_time_feat(train_merged)
  test_merged = additional_time_feat(test_merged)
  train_merged,le_1= additional_cat_feat(train_merged)
  test_merged['Type_le'] = le_1.transform(test_merged['Type'])
  train_merged = additional_month_feat_train(train_merged, col='Weekly_Sales')
  test_merged = additional_month_feat_test(train_merged, test_merged, col='Weekly_Sales')

  mlflow.log_param("time_features_added", ["Day", "Week", "Month", "Year", "Month_sin", "Month_cos", "Week_sin", "Week_cos"])
  mlflow.log_param("rolling_windows", [4, 8, 12])
  mlflow.log_param("categorical_encoding", "LabelEncoder")


  print(f"Final train shape is : {train_merged.shape}")
  print(f"Final test shape is : {test_merged.shape}")

  mlflow.log_metric("final_train_features", train_merged.shape[1])
  mlflow.log_metric("final_test_features", test_merged.shape[1])

# **Pipeline Creation**

In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class WalmartDataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.label_encoder = None
        self.train_data_for_rolling = None

    def fit(self, X, y=None):
        self.label_encoder = le_1
        self.train_data_for_rolling = train_merged

        return self

    def transform(self, X):
        X_processed = X.copy()
        X_processed = X_processed.merge(stores_df, on='Store', how='left')
        X_processed = X_processed.merge(features_df, on=['Store', 'Date'], how='left')


        if 'IsHoliday_x' in X_processed.columns and 'IsHoliday_y' in X_processed.columns:
            X_processed['IsHoliday'] = X_processed['IsHoliday_y'].fillna(X_processed['IsHoliday_x'])
            X_processed = X_processed.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)


        X_processed = object_to_datetime(X_processed, col='Date', curr_part='Pipeline transform')


        num_cols = ['CPI', 'Fuel_Price', 'Unemployment', 'Temperature']
        fill_numericals(num_cols, X_processed, X_processed)  # Just pass same df twice


        markdowns = [col for col in X_processed.columns if 'MarkDown' in col]
        for col in markdowns:
            X_processed[col] = X_processed[col].fillna(0)
        X_processed['IsHoliday'] = X_processed['IsHoliday'].fillna(False)
        removed_cols = ['MarkDown1', 'MarkDown3', 'MarkDown5']
        X_processed = X_processed.drop([col for col in removed_cols if col in X_processed.columns], axis=1)
        X_processed = additional_time_feat(X_processed)
        X_processed['Type_le'] = self.label_encoder.transform(X_processed['Type'])
        X_processed = additional_month_feat_test(self.train_data_for_rolling, X_processed, col='Weekly_Sales')
        cols_to_remove = ['Date', 'Type']
        if 'Weekly_Sales' in X_processed.columns:
            cols_to_remove.append('Weekly_Sales')
        X_processed = X_processed.drop([col for col in cols_to_remove if col in X_processed.columns], axis=1)

        return X_processed

# **Cross Validation**

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
def run_cross_validation(X_train, y_train):
    """Complete your cross validation function"""

    with mlflow.start_run(run_name="XGBoost_Cross_Validation") as run:
        tscv = TimeSeriesSplit(n_splits=5)
        cv_scores = []

        base_params = {
            'objective': 'reg:squarederror',
            'max_depth': 6,
            'learning_rate': 0.1,
            'n_estimators': 1000,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'n_jobs': -1
        }
        for param, value in base_params.items():
            mlflow.log_param(f"cv_{param}", value)

        for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
            print(f"Working on fold {fold + 1}/5")

            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model = xgb.XGBRegressor(**base_params)
            model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

            y_pred = model.predict(X_val)
            mae = mean_absolute_error(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            r2 = r2_score(y_val, y_pred)

            cv_scores.append({'fold': fold, 'mae': mae, 'rmse': rmse, 'r2': r2})

            # Log metrics
            mlflow.log_metric(f"fold_{fold}_mae", mae)
            mlflow.log_metric(f"fold_{fold}_rmse", rmse)
            mlflow.log_metric(f"fold_{fold}_r2", r2)

            print(f"  MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}")
        avg_mae = np.mean([score['mae'] for score in cv_scores])
        avg_rmse = np.mean([score['rmse'] for score in cv_scores])
        avg_r2 = np.mean([score['r2'] for score in cv_scores])

        mlflow.log_metric("cv_avg_mae", avg_mae)
        mlflow.log_metric("cv_avg_rmse", avg_rmse)
        mlflow.log_metric("cv_avg_r2", avg_r2)

        print(f"\\nCross Validation Results:")
        print(f"Average MAE: {avg_mae:.2f}")
        print(f"Average RMSE: {avg_rmse:.2f}")
        print(f"Average R2: {avg_r2:.4f}")

        return cv_scores

# **Training**

In [ ]:
def run_final_training(X_train, y_train):
    """Complete your final training function"""

    with mlflow.start_run(run_name="XGBoost_Final_Training") as run:

        final_params = {
            'objective': 'reg:squarederror',
            'max_depth': 6,
            'learning_rate': 0.1,
            'n_estimators': 1000,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'n_jobs': -1
        }


        for param, value in final_params.items():
            mlflow.log_param(param, value)


        final_model = xgb.XGBRegressor(**final_params)
        final_model.fit(X_train, y_train)


        pipeline = Pipeline([
            ('preprocessor', WalmartDataPreprocessor()),
            ('model', xgb.XGBRegressor(**final_params))
        ])
        original_X = train_df.drop('Weekly_Sales', axis=1)
        original_y = train_df['Weekly_Sales']
        pipeline.fit(original_X, original_y)
        mlflow.sklearn.log_model(
            pipeline,
            "xgboost_pipeline",
            registered_model_name="walmart_xgboost_pipeline"
        )

        mlflow.xgboost.log_model(
            final_model,
            "xgboost_model",
            registered_model_name="walmart_xgboost_model"
        )
        #mlflow.sklearn.log_model(pipeline, "xgboost_pipeline")
        #mlflow.xgboost.log_model(final_model, "xgboost_model")


        train_pred = final_model.predict(X_train)
        mae = mean_absolute_error(y_train, train_pred)
        rmse = np.sqrt(mean_squared_error(y_train, train_pred))
        r2 = r2_score(y_train, train_pred)
        mlflow.log_metric("final_mae", mae)
        mlflow.log_metric("final_rmse", rmse)
        mlflow.log_metric("final_r2", r2)

        print(f"Final model metrics:")
        print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}")

        return pipeline, final_model

# **Main Execution**

In [ ]:

feature_cols = [col for col in train_merged.columns if col not in ['Weekly_Sales', 'Date', 'Type']]
removed_cols = ['MarkDown1', 'MarkDown3', 'MarkDown5']
feature_cols = [col for col in feature_cols if col not in removed_cols]

X_train = train_merged[feature_cols].copy()
y_train = train_merged['Weekly_Sales'].copy()

print(f"\\nTraining with {len(feature_cols)} features using your processed data")
print(f"Training data shape: {X_train.shape}")

print("RUNNING CROSS VALIDATION")

cv_scores = run_cross_validation(X_train, y_train)
print("TRAINING FINAL MODEL")
pipeline, model = run_final_training(X_train, y_train)

print("GENERATING PREDICTIONS")


test_feature_cols = [col for col in feature_cols if col in test_merged.columns]
X_test_processed = test_merged[test_feature_cols].copy()
for col in feature_cols:
    if col not in X_test_processed.columns:
        print(f"Adding missing column {col} with default value 0")
        X_test_processed[col] = 0
X_test_processed = X_test_processed[feature_cols]
test_predictions = model.predict(X_test_processed)

print(f"Generated {len(test_predictions)} test predictions")
print(" XGBoost model complete with pipeline!")

# **Submission File**

In [ ]:
with mlflow.start_run(run_name="XGBoost_Submission_Generation") as run:
    print("\\n" + "="*50)
    print("CREATING SUBMISSION FILE")
    print("="*50)
    test_submission = test_merged[['Store', 'Dept', 'Date']].copy()
    test_submission['Weekly_Sales'] = test_predictions
    test_submission['Id'] = (test_submission['Store'].astype(str) + '_' +
                           test_submission['Dept'].astype(str) + '_' +
                           test_submission['Date'].dt.strftime('%Y-%m-%d'))

    submission = test_submission[['Id', 'Weekly_Sales']].copy()
    submission_path = f"{FOLDERNAME}/xgboost_submission.csv"
    submission.to_csv(submission_path, index=False)


    mlflow.log_artifact(submission_path)
    mlflow.log_metric("submission_rows", len(submission))

    print(f"Submission saved to: {submission_path}")
    print(f"Submission shape: {submission.shape}")
    print("\\nFirst 10 rows of submission:")
    print(submission.head(10))

    print(" XGBOOST EXPERIMENT COMPLETED SUCCESSFULLY!")
    print(" All MLflow runs logged")
    print(" Model registered in Model Registry")
    print(" Pipeline ready for deployment")
    print(" Submission file generated")